## Query Performance in Snowflake

## Snowflake Data Objects

In [ ]:
CREATE OR REPLACE MATERIALIZED VIEW top_customers AS 
SELECT o.o_custkey AS customer_id,   
c.c_name AS customer_name, 
CASE 
WHEN SUM(o.o_totalprice) > 5000000 THEN 'Over Price' 
WHEN SUM(o.o_totalprice) > 3000000 THEN 'Top Price' 
WHEN SUM(o.o_totalprice) > 2000000 THEN 'Average Price' 
ELSE 'Review Price' 
END AS total_price 
FROM customer AS c 
JOIN orders AS o ON c.c_custkey = o.o_custkey 
GROUP BY o.o_custkey, c.c_name 
HAVING total_price > 2000000;

### Implementing views
The Star Customer financial company seeks to streamline its data management to efficiently categorize customers and track their product usage. You will create a view to facilitate quick and efficient data retrieval, calculating specific customer categories.

In [ ]:
-- Create a view customer_financial_summary
CREATE OR REPLACE VIEW customer_financial_summary AS 
SELECT c.customerid, 
	c.estimatedsalary,
	cp.productid
FROM customers AS c
	-- Merge entity
LEFT JOIN customerproducts AS cp 
ON c.customerid = cp.customerid;

In [ ]:
CREATE OR REPLACE VIEW customer_financial_summary AS
SELECT c.customerid, 
	-- Create a new conditional attribute
    CASE 
        WHEN AVG(c.estimatedsalary) > 150000 THEN 'Top Income'
        WHEN AVG(c.estimatedsalary) > 90000 THEN 'High Income'
        WHEN AVG(c.estimatedsalary) > 20000 THEN 'Average Income'
        ELSE 'Low Income'
    END AS customer_category,
    -- Add aggregation to the attributes
    COUNT(DISTINCT cp.productid) AS product_count
FROM customers AS c
	LEFT JOIN customerproducts AS cp 
	ON c.customerid = cp.customerid
-- Group the results
GROUP BY c.customerid;

## Query Optimization

In [ ]:
# Subqueries 
-- Query all guests that have more than 1000 loyalty points 
SELECT * 
FROM guests
WHERE id IN (SELECT guest_id 
             FROM loyalty_program 
             WHERE loyalty_points > 1000
             );

In [ ]:
WITH latest_booking AS (
    SELECT guest_id, 
          MAX(checkout_date) AS latest_checkout 
    FROM booking_details 
    GROUP BY guest_id
    ) 
SELECT bd.*,   
       bd.checkout_date AS latest_booking_date 
FROM booking_details bd 
    JOIN latest_booking lb 
        ON bd.guest_id = lb.guest_id 
        AND bd.checkout_date = lb.latest_checkout;

### Subquery mastery
You are the data modeler of a business that manages hotel chain data. The business team has the task of examining customer data to inform retention strategies. The marketing department needs insights into high-income customers' activity status and engagement with the hotel's services. Your goal is to use the existing customer data to identify active and churned customers with a high estimated salary and to analyze their average age and tenure. To achieve this goal, you can use this pre-built common table expression CustomerStatus.

In [ ]:
WITH customer_status AS (
	SELECT c.customerid,
  		c.age,
        c.tenure,
        CASE 
            WHEN ch.customerid IS NOT NULL THEN 'Churned' 
            ELSE 'Active' 
        END AS status
    FROM customers AS c
    	LEFT JOIN churn AS ch 
  		ON c.customerid = ch.customerid
    GROUP BY c.customerid, c.age, c.tenure, status
)
-- Extract attribute from CTE
SELECT status
FROM customer_status
-- Filter results
WHERE customerid IN (SELECT customerid
                    FROM customers
                   WHERE estimatedsalary > 175000);

In [ ]:
WITH customer_status AS (
	SELECT c.customerid,
  		c.age,
        c.tenure,
        CASE 
            WHEN ch.customerid IS NOT NULL THEN 'Churned' 
            ELSE 'Active' 
        END AS status
    FROM customers AS c
    	LEFT JOIN churn AS ch 
  		ON c.customerid = ch.customerid
    GROUP BY c.customerid, c.age, c.tenure, status
)
SELECT status,
	-- Count customers
	COUNT(customerid) AS unique_customers
FROM customer_status
WHERE customerid IN (SELECT customerid 
                     FROM customers 
                     WHERE estimatedsalary > 175000)
-- Aggregate values
GROUP BY status;

In [ ]:
WITH customer_status AS (
	SELECT c.customerid,
  		c.age,
        c.tenure,
        CASE 
            WHEN ch.customerid IS NOT NULL THEN 'Churned' 
            ELSE 'Active' 
        END AS status
    FROM customers AS c
    	LEFT JOIN churn AS ch 
  		ON c.customerid = ch.customerid
    GROUP BY c.customerid, c.age, c.tenure, status
)
SELECT status,
	COUNT(customerid) AS unique_customers,
    -- Calculate averages
    AVG(age) AS average_age,
    AVG(tenure) AS average_tenure
FROM customer_status
WHERE customerid IN (SELECT customerid 
                     FROM customers 
                     WHERE estimatedsalary > 175000)
GROUP BY status
-- Filter data
HAVING average_tenure > 2;